In [78]:
import boto3
session = boto3.Session(profile_name='corpdeveu')
client = session.client('elbv2')

In [79]:
elb_dict = client.describe_load_balancers(Names=['vbo-devb-redem-rest-api-elb'])

Enter MFA code for arn:aws:iam::753764768369:mfa/joshua.armitage: ········


In [92]:
elb = elb_dict['LoadBalancers'][0]
elb_arn = elb['LoadBalancerArn']

In [81]:
listeners_dict = client.describe_listeners(LoadBalancerArn=elb_arn)

In [ ]:
listeners_dict['Listeners']

In [82]:
listener = [listener for listener in listeners_dict['Listeners'] if listener['Port'] == 443]

In [83]:
listener_arn = listener[0]['ListenerArn']

In [84]:
rules_dict = client.describe_rules(ListenerArn=listener_arn)

In [91]:
tg = {}
tg['name'] = 'JoshsSpecialTargetGroup'
tg['protocol'] = 'HTTPS'
tg['port'] = 443
tg['vpcid'] = elb['VpcId']

In [112]:
response = client.create_target_group(
    Name=tg['name'],
    Protocol=tg['protocol'],
    Port=tg['port'],
    VpcId=tg['vpcid']
)
response

In [113]:
target_group_arns = [group['TargetGroupArn'] for group in response['TargetGroups']]
target_group_arns

['arn:aws:elasticloadbalancing:eu-west-1:379807770492:targetgroup/JoshsSpecialTargetGroup/a7d0ce797394b950']

In [106]:
def delete_target_groups():
    for arn in target_group_arns:
        client.delete_target_group(
            TargetGroupArn=arn
        )

In [107]:
rules_dict

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '858',
   'content-type': 'text/xml',
   'date': 'Wed, 14 Feb 2018 06:25:54 GMT',
   'x-amzn-requestid': 'e91c88a9-114f-11e8-a072-c370ea171abd'},
  'HTTPStatusCode': 200,
  'RequestId': 'e91c88a9-114f-11e8-a072-c370ea171abd',
  'RetryAttempts': 0},
 u'Rules': [{u'Actions': [{u'TargetGroupArn': 'arn:aws:elasticloadbalancing:eu-west-1:379807770492:targetgroup/RedemrestapiTarget2GSbzu/89ea4ff8574532c9',
     u'Type': 'forward'}],
   u'Conditions': [],
   u'IsDefault': True,
   u'Priority': 'default',
   u'RuleArn': 'arn:aws:elasticloadbalancing:eu-west-1:379807770492:listener-rule/app/vbo-devb-redem-rest-api-elb/645137812a2d8c64/917680190e67e421/502f592d45429d4d'}]}

In [114]:
rule = {}
rule['ListenerArn'] = listener_arn
rule['Conditions'] = [{
    'Field': 'path-pattern',
    'Values': ['/redemptions/*']
}]
rule['Priority'] = 1
rule['Actions'] = [{'Type': 'forward', 'TargetGroupArn': target_group_arns[0]}]

In [115]:
create_rule_response = client.create_rule(
    ListenerArn=rule['ListenerArn'],
    Conditions=rule['Conditions'],
    Priority=rule['Priority'],
    Actions=rule['Actions']
)
create_rule_response

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '1029',
   'content-type': 'text/xml',
   'date': 'Wed, 14 Feb 2018 06:54:14 GMT',
   'x-amzn-requestid': 'de9e15a2-1153-11e8-9d56-395051d6b796'},
  'HTTPStatusCode': 200,
  'RequestId': 'de9e15a2-1153-11e8-9d56-395051d6b796',
  'RetryAttempts': 0},
 u'Rules': [{u'Actions': [{u'TargetGroupArn': 'arn:aws:elasticloadbalancing:eu-west-1:379807770492:targetgroup/JoshsSpecialTargetGroup/a7d0ce797394b950',
     u'Type': 'forward'}],
   u'Conditions': [{u'Field': 'path-pattern', u'Values': ['/redemptions/*']}],
   u'IsDefault': False,
   u'Priority': '1',
   u'RuleArn': 'arn:aws:elasticloadbalancing:eu-west-1:379807770492:listener-rule/app/vbo-devb-redem-rest-api-elb/645137812a2d8c64/917680190e67e421/ed0055f1dbb02c0a'}]}

In [116]:
def delete_rule():
    delete_rule_response = client.delete_rule(
        RuleArn=create_rule_response['Rules'][0]['RuleArn']
    )